In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib inline
from PhysicsDatasets import SHODatasetXV, DampedSHODatasetXV, DampedSHODatasetV2
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import json
from matplotlib.colors import LogNorm, Normalize
from utils import load_model
import utils
import os
from datetime import datetime
import yaml
import re
from tqdm import tqdm
import plotting as ptools
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
models = ["tokenized_discrete_mkb_mAll_kAll_betaAll"]#,"tokenized_discrete_mkb_mLo_kAll_betaAll","tokenized_discrete_mkb_mHi_kAll_betaAll","tokenized_discrete_mkb_mLoHi_kAll_betaAll"]

data = {}
for m in models:
    model_dir = ptools.get_latest_model_dir(m)
    results = [f for f in os.listdir(model_dir) if "iter" in f and ".npy" in f]
    n_iters = [int(re.search("iter(\d+)",m).group(1)) for m in results]
    data[m] = {}
    for n in n_iters:
        data[m][n] = np.load(f"{model_dir}/eval_iter{n}.npy")
    masses = np.load(f"{model_dir}/eval_masses.npy")
    ks = np.load(f"{model_dir}/eval_ks.npy")
    betas = np.load(f"{model_dir}/eval_betas.npy")

"""nrow = 4
ncol = len(masses)//4 + 1*int(len(masses)%4 > 0)
fig,axes = plt.subplots(4,ncol,figsize=(ncol*8,nrow*6))
beta_test = 0
ib = list(betas).index(beta_test)
n_ctx_test = 50

for i,m_test in enumerate(masses):
    plt.sca(axes.flatten()[i])
    im = list(masses).index(m_test)
    cmap = plt.get_cmap('viridis')
    colors = [cmap(i / (len(iter_ckpts) - 1)) for i in range(len(iter_ckpts))]

    plt.plot(ks,loss_best[n_ctx_test-1,im,:,ib],color='red')
    for ii,n_iter in enumerate(iter_ckpts):
        plt.plot(ks,loss_byIter[n_iter][n_ctx_test-1,im,:,ib],color=colors[ii])
    #plt.yscale('log')
    plt.title(f"M = {m_test}")"""

'nrow = 4\nncol = len(masses)//4 + 1*int(len(masses)%4 > 0)\nfig,axes = plt.subplots(4,ncol,figsize=(ncol*8,nrow*6))\nbeta_test = 0\nib = list(betas).index(beta_test)\nn_ctx_test = 50\n\nfor i,m_test in enumerate(masses):\n    plt.sca(axes.flatten()[i])\n    im = list(masses).index(m_test)\n    cmap = plt.get_cmap(\'viridis\')\n    colors = [cmap(i / (len(iter_ckpts) - 1)) for i in range(len(iter_ckpts))]\n\n    plt.plot(ks,loss_best[n_ctx_test-1,im,:,ib],color=\'red\')\n    for ii,n_iter in enumerate(iter_ckpts):\n        plt.plot(ks,loss_byIter[n_iter][n_ctx_test-1,im,:,ib],color=colors[ii])\n    #plt.yscale(\'log\')\n    plt.title(f"M = {m_test}")'